## Import the necessary modules

In [1]:
import myNHLstats as mns
from myNHLstats import GameStats, GameMap, PlotGameStats
from nhlstats import *

import pandas as pd
import datetime as dt
from datetime import *

import ipywidgets as widgets
from ipywidgets import *
from IPython.display import clear_output

from IPython.display import display, Javascript

disable_js = """
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}
"""
display(Javascript(disable_js))

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

<IPython.core.display.Javascript object>

## Import module needed to publish reports using/to datapane

In [2]:
#import datapane as dp
#from datetime import date

#dp.login(token="")

## Create widgets

In [3]:
desc = "\n\nAuthor's Notes:\n\n" + "- Using Plotly's `subplots` & `graph_objects` class's (python), it is currently only possible to attach a legend " + \
        "in the first subplot. This makes it impossible to add legends to the maps, momentum, or P5 plots :(. But, all of the plots are " + \
        "interactive and moving your cursor over the plot will bring up labels for the data points. I know its possible to make this happen " + \
        "but am currently researching how to make it work using Plotly.\n\n- The circular, pie-chart-esque plots are called `sunburst` plots. " + \
        "I prefer to call them `Puck Plots`. Click on different parts of the plot to focus the plot on that element and it's corresponding data. " + \
        "Some of these puck plots are restricted to displaying a max depth of 2 or 3 levels, but those plots have more info to show when you " + \
        " focus in on a specific element.\n\n- `Net Momentum` is a _very subjective_ measure of which team is controlling the play. " + \
        "It is calculated as the sum product of goals, shots, shot attempts, and hits over the past 5 minutes (for any point of time in the game) " + \
        "multiplied by arbitrary weights for the respective stats. Current weights for momentum are Goal: 10, Shot: 5, Hit: 3, Shot Attempt: 1. " + \
        "Momentum is calculated for each team and then the Away team's momentum is subtracted from the Home team's momentum to get the Net Momentum. " + \
        "A positive Net Momentum means the Home team is in theory 'dominating' the play at that point, and a negative Net Momentum implies the same " + \
        "for the Away team.\n\n- In the future, I plan to refine the weights used to calculate momentum using various regressions and larger " + \
        "data sets. The goal of Net Momentum is to have a single measure of the statistical likelihood for a team scoring a goal. My " + \
        "hypothesis is that goals scored by lower skilled teams will correspond to a higher Net Momentum on average than for a higher skilled team. " + \
        "I also plan to refine the weighting system to account for how long ago each event happened. For example, a shot that occured 4 minutes " + \
        "ago should not have the same weight as a shot that happened 1 minute ago, but they currently have the same influence on team momentum."

#print(desc)

In [4]:
style = {'description_width': 'initial'}

DatePicker = widgets.DatePicker(
    description = 'Pick a Date',
    disabled    = False,
    value       = date.today(),
    style       = style
)
HomeColor       = widgets.Checkbox(
    value       = False,
    description = 'Alternate Home Color:',
    disabled    = False,
    indent      = False
)

AwayColor       = widgets.Checkbox(
    value       = False,
    description = 'Alternate Away Color:',
    disabled    = False,
    indent      = False
)

chkPublish      = widgets.Checkbox(
    value       = False,
    description = 'Publish Report to DataPane:',
    disabled    = False,
    indent      = False
)

templates = ["plotly", "plotly_white", "plotly_dark", "ggplot2", "seaborn", "simple_white", "none"]
drpTemplate = widgets.Dropdown(options     = templates, 
                               value       = 'plotly_white',
                               description = 'Plotly Template:',
                               disabled    = False, 
                               style       = style)
def lst_unique(x):
    return list(dict.fromkeys(x))

def SelectGames(GameDay):
    # Download games index for selected gameday to pandas DF
    try:
        games = pd.DataFrame(list_games(GameDay, GameDay))
        games = games.drop(columns=['season'])
    except:
        team_select = widgets.Dropdown(options     = ["NO GAMES"],
                                       value       = "NO GAMES",
                                       description = 'Team Name: ',
                                       disabled    = False,
                                       style       = style
                                      )
        clear_output(wait=True)
        return None, team_select
    
    if len(games) > 0:
        #Create lists for all teams playing, and all game_id's
        teams = []
        [teams.append(i) for i in games.away_team]
        [teams.append(i) for i in games.home_team]

        teams.sort()
        teams = lst_unique(teams)
    
    team_select = widgets.Dropdown(options     = teams,
                                   value       = teams[0],
                                   description = 'Team Name: ',
                                   disabled    = False,
                                   style       = style)
        
    return games, team_select

goButton = widgets.Button(value        = False,
                          description  = 'Create Plots',
                          disabled     = False,
                          button_style = 'success', # 'success', 'info', 'warning', 'danger' or ''
                          tooltip      = 'Description',
                          icon         = 'check', # (FontAwesome names without the `fa-` prefix)
                          style        = style)
def click(b):
    
    gameid, Home, Away = games.loc[(games.home_team == team_select.value) | \
                                   (games.away_team == team_select.value), 
                                   ['game_id','home_team','away_team']].max()
    clear_output(wait=False)
    display(goButton)
    
    # Create game plot
    gp = PlotGameStats(gameid    = gameid, 
                       HomeColor = HomeColor.value, 
                       AwayColor = AwayColor.value)
    
    gp.go(template=drpTemplate.value)
    fig = gp.fig

    Folder = r"C:\Users\grega\Google Drive\Python\NHLstats\Live NHL Stats\HTML Files\\"
    fig.write_html(f"{Folder}{Away}_at_{Home}_{gp.gameDay}.html")
    
    fig.show()
    
    if chkPublish.value == True:
        # Create report
        r = dp.Report(f"{Away} @ {Home} | {gp.gameDay}",
                      f'_Built using data from the NHL Stats API on {date.today()}_'  + desc,
                      fig,
                      gp.plays,
                )

        # Publish
        r.publish(name=f"{Away} @ {Home} | {gp.gameDay}", 
                  open=True, 
                  description=f'Built using data from the NHL Stats API on {date.today()}')

goButton.on_click(click)    
games, team_select = SelectGames(DatePicker.value)


## Display game plot/map generator

In [5]:
@interact
def change(DP=DatePicker, 
           ts=team_select,
           HC=HomeColor, 
           AC=AwayColor,
           PB=chkPublish,
           DT=drpTemplate):
    global games
    games, temp = SelectGames(DatePicker.value)
    
    try:
        teams = []
        [teams.append(i) for i in games.away_team]
        [teams.append(i) for i in games.home_team]
    
        teams.sort()
        teams = lst_unique(teams)
        team_select.options = teams
    
    except:
        team_select.options =['NO GAMES']
        
    clear_output(wait=True)
    display(games)
    display(goButton)

interactive(children=(DatePicker(value=datetime.date(2021, 2, 28), description='Pick a Date', style=Descriptio…